In [1]:
from __future__ import division
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os, glob
import sys
import numpy as np
import cv2
import tensorflow as tf
import keras
import mrcnn.config
import mrcnn.utils
#import keras.backend as
from mrcnn.model import MaskRCNN
from pathlib import Path
from twilio.rest import Client
import skimage.io
import random 

# from moviepy.editor import VideoFileClip
import imageio
cwd = os.getcwd()

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Using TensorFlow backend.


## Mask RCNN Configuration

In [2]:
class MaskRCNNConfig(mrcnn.config.Config):
    NAME = "coco_pretrained_model_config"
    IMAGES_PER_GPU = 1
    GPU_COUNT = 1
# COCO dataset has 80 classes (1 background class)    
    NUM_CLASSES = 1 + 80  
    DETECTION_MIN_CONFIDENCE = 0.6

## COCO filter
        

In [3]:
# Filter a list of Mask R-CNN detection results to get only the detected cars / trucks
def get_car_boxes(boxes, class_ids):
    car_boxes = []

    for i, box in enumerate(boxes):
# If the detected object isn't a bicycle/car/motorcycle/bus/truck, skip it
        if class_ids[i] in [2, 3, 4, 6, 8]:
            car_boxes.append(box)

    return np.array(car_boxes)

## Directories/Path/load/model

In [4]:
# Root directory of the project
ROOT_DIR = Path(".")

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    mrcnn.utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

# Video file or camera to process - set this to 0 to use your webcam instead of a video file
VIDEO_SOURCE = "parking.mp4"

# Create a Mask-RCNN model in inference mode
model = MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=MaskRCNNConfig())

# Load pre-trained model
model.load_weights(COCO_MODEL_PATH, by_name=True)

# Location of parking spaces
parked_car_boxes = None

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [5]:
# Load the video file we want to run detection on
video_capture = cv2.VideoCapture(VIDEO_SOURCE)
#video_capture.set(cv2.CAP_PROP_FPS, 0.1)

#fps = FPS().start()
# How many frames of video we've seen in a row with a parking space open
free_space_frames = 0

# Have we sent a notification alert yet?
notification_sent = False

frame_num = 0

# Loop over each frame of video
while video_capture.isOpened():
    success, frame = video_capture.read()
    if not success:
        break

    print(frame_num)
    # Increasing the speed of video to be shown on the screen
    video_capture.set(cv2.CAP_PROP_POS_MSEC, frame_num*1000)
    print("Position : %d" % video_capture.get(cv2.CAP_PROP_POS_MSEC))
    frame_num += 1
    
    # Convert the image from BGR (used in open CV) to RGB
    rgb_image = frame[:, :, ::-1]

    # Run the image through the Mask R-CNN model to get results.
    results = model.detect([rgb_image], verbose=0)

    # Mask R-CNN assumes we are running detection on multiple images.
    # Here we take just one result (one image/frame)
    r = results[0]

    # The r variable will now have the results of detection:
    # - r['rois'] are the bounding box of each detected object
    # - r['class_ids'] are the class id (type) of each detected object
    # - r['scores'] are the confidence scores for each detection
    # - r['masks'] are the object masks for each detected object (which gives you the object outline)

    if parked_car_boxes is None:
        # This is the first frame of video - assume all the cars detected are in parking spaces.
        # Save the location of each car as a parking space box and go to the next frame of video.
        parked_car_boxes = get_car_boxes(r['rois'], r['class_ids'])
    else:
        # We already know where the parking spaces are. Check if any are currently unoccupied.

        # Get where cars are currently located in the frame
        car_boxes = get_car_boxes(r['rois'], r['class_ids'])

        # See how much those cars overlap with the known parking spaces
        overlaps = mrcnn.utils.compute_overlaps(parked_car_boxes, car_boxes)

        # Assume no spaces are free until we find one that is free
        free_space = False

        # Loop through each known parking space box
        for parking_area, overlap_areas in zip(parked_car_boxes, overlaps):

            # For this parking space, find the max amount it was covered by any
            # car that was detected in our image (doesn't really matter which car)
            max_IoU_overlap = np.max(overlap_areas)

            # Get the top-left and bottom-right coordinates of the parking area
            y1, x1, y2, x2 = parking_area

            # Check if the parking space is occupied by seeing if any car overlaps
            # it by more than 0.15 using IoU
            if max_IoU_overlap < 0.15:
                # Parking space not occupied! Draw a green box around it
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                # Flag that we have seen at least one open space
                free_space = True
            else:
                # Parking space is still occupied - draw a red box around it
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 1)

            # Write the IoU measurement inside the box
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, f"{max_IoU_overlap:0.2}", (x1 + 6, y2 - 6), font, 0.3, (255, 255, 255))

        # If at least one space was free, start counting frames
        # This is so we don't alert based on one frame of a spot being open.
        # This helps prevent the script triggered on one bad detection.
        if free_space:
            free_space_frames += 1
        else:
            # If no spots are free, reset the count
            free_space_frames = 0

        # If a space has been free for several frames, we are pretty sure it is really free!
        if free_space_frames > 2:
            # Write SPACE AVAILABLE!! at the top of the screen
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, f"SPACE IS AVAILABLE!", (10, 150), font, 3.0, (0, 255, 0), 2, cv2.FILLED)

            # If we haven't sent a Notification yet, send it!
            if not notification_sent:
                print("SENDING NOTIFICATION!")
                #message = client.messages.create(
                #    body="Parking space open - go go go!",
                #    from_=twilio_phone_number,
                #    to=destination_phone_number
                #)
                notification_sent = True
###Use tf.cast instead SENDING Notification!!!
        # Show the frame of video on the screen
        cv2.imshow('Video', frame)

    # Hit 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up everything when finished
video_capture.release()
cv2.destroyAllWindows()

0
Position : 0
1
Position : 1000
2
Position : 2000
3
Position : 3000
4
Position : 4000
5
Position : 5000
6
Position : 6000
7
Position : 7000
SENDING NOTIFICATION!
8
Position : 8000
9
Position : 9000
10
Position : 10000
11
Position : 11000
12
Position : 12000
13
Position : 13000
14
Position : 14000
15
Position : 15000
16
Position : 16000
17
Position : 17000
18
Position : 17680
